In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

### 기업 정보 버튼 누르기

In [30]:
prefix = "https://www.saramin.co.kr/zf_user/search?keydownAccess=&searchword="
searchword = "로봇 SW 개발"
url = prefix + searchword.replace(" ", "+")
driver = webdriver.Chrome(service=Service("/home/koo4802/dev_ws/eda/driver/chromedriver"))
driver.set_window_size(1400, 1080)

for _ in range(2):
    driver.get(url)
    time.sleep(1)
company_link = driver.find_element(By.CSS_SELECTOR, ".tab_search_result.on li a[target='company']")
company_link.click()


### 기업 링크 리스트 추출

In [31]:
href_list = [comany_link.get_attribute("href") for comany_link in driver.find_elements(By.CSS_SELECTOR, ".corp_name a")]
href_list

['https://www.saramin.co.kr/zf_user/company-info/view?csn=U1RmUDhEQ2tTb0dGZXNOSnB5clp4dz09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=aTc1RVBIZklBZWU4Tzk1VGFWeFFIZz09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=bW9CL2JQZFQ5N2V3UFU1VWo1NWdBUT09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=VWx6di93RlR4RDZpTWhWYVVURXFYdz09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=QmZkeUFWZzFhVXRyRXVOakxYeFpQUT09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=VStOT3crRTBmS1lZRFBHM3Y3Uk1rZz09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=ZEhCa0xrUEdxbXlZbGpKYXJscGpLZz09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=cDNlNzFteTM1N1pZTmJMNlh6WUY2UT09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=VFBNSkFGVXRBc0k2aTB1b3p5OWUyZz09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=cllYYkEyQmYwQThCL1YrYThtQmNNdz09',
 'https://www.saramin.co.kr/zf_user/company-info/view?csn=YVNrdUNONDNZ

### URL 타고 넘어가서 기업 정보 추출

In [72]:
driver.get(href_list[1])
title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
key_info = driver.find_elements(By.CLASS_NAME, "company_summary_desc")
for info in key_info:
    print(info.text)


2017년 5월 8일 설립
기업형태
사원수
매출액


In [63]:
driver.get(href_list[0])
title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
key_info = driver.find_elements(By.CLASS_NAME, "company_summary_desc")
company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
company_details = driver.find_elements(By.CLASS_NAME, "company_details_group dd")
# 기본 정보 설정
company_info = {"기업명": title}

for i, form_element in enumerate(company_form):
    if i == 0 and " " in form_element.text:  # 첫 번째 요소가 "업력 10년" 같은 형식인 경우
        parts = form_element.text.split(" ", 1)
        if len(parts) >= 2:
            company_info[parts[0]] = parts[1]
        else:
            company_info[f"정보{i+1}"] = form_element.text
    elif i < len(key_info):  # key_info에 해당하는 인덱스가 있는 경우
        company_info[key_info[i].text] = form_element.text
    else:  # key_info에 해당하는 인덱스가 없는 경우
        company_info[f"정보{i+1}"] = form_element.text
company_info

{'기업명': '(주)로보티즈',
 '업력': '업력 27년차',
 '업종': '그 외 기타 특수목적용 기계 제조업',
 '기업형태': '중소기업',
 '사원수': '158 명',
 '매출액': '비공개',
 '주소': '서울 강서구 마곡중앙5로1길 37'}

### 연봉 정보(최소, 평균, 최대)

In [64]:
menu_ul_element = driver.find_element(By.CLASS_NAME, "menu_list")
li_list = menu_ul_element.find_elements(By.TAG_NAME, "li")
for li in li_list:
    if li.text == "연봉정보":
        li.click()
        break
avg_salary = driver.find_element(By.CLASS_NAME, "average_currency").text
min_salary = driver.find_element(By.CLASS_NAME, "min_txt").text
max_salary = driver.find_element(By.CLASS_NAME, "max_txt").text
company_info["연봉정보"] = {"최소" : min_salary, "평균" : avg_salary, "최대" : max_salary}
company_info

{'기업명': '(주)로보티즈',
 '업력': '업력 27년차',
 '업종': '그 외 기타 특수목적용 기계 제조업',
 '기업형태': '중소기업',
 '사원수': '158 명',
 '매출액': '비공개',
 '주소': '서울 강서구 마곡중앙5로1길 37',
 '연봉정보': {'최소': '3,130만원', '평균': '5,406 만원', '최대': '6,948만원'}}

### 셀 하나로 병합

In [67]:
company_info_list = []
for href in href_list[:3]:
    driver.get(href)
    time.sleep(1)
    title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
    company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
    company_details = driver.find_elements(By.CLASS_NAME, "company_details_group dd")
    company_info = {"기업명" : title,
                    "업력" : company_form[0].text,
                    "업종" : company_details[0].text,
                    "기업형태" : company_form[1].text,
                    "사원수" : company_form[2].text if len(company_form) > 2 else "비공개",
                    "매출액" : company_form[3].text if len(company_form) > 3 else "비공개",
                    "주소" : company_details[4].text.split("\n")[0]}
    menu_ul_element = driver.find_element(By.CLASS_NAME, "menu_list")
    li_list = menu_ul_element.find_elements(By.TAG_NAME, "li")
    for li in li_list:
        if li.text == "연봉정보":
            li.click()
            break
    avg_salary = driver.find_element(By.CLASS_NAME, "average_currency").text
    min_salary = driver.find_element(By.CLASS_NAME, "min_txt").text
    max_salary = driver.find_element(By.CLASS_NAME, "max_txt").text
    company_info["연봉정보"] = {"최소" : min_salary, "평균" : avg_salary, "최대" : max_salary}
    company_info_list.append(company_info)
company_info_list

[{'기업명': '(주)로보티즈',
  '업력': '업력 27년차',
  '업종': '그 외 기타 특수목적용 기계 제조업',
  '기업형태': '중소기업',
  '사원수': '158 명',
  '매출액': '비공개',
  '주소': '서울 강서구 마곡중앙5로1길 37',
  '연봉정보': {'최소': '3,130만원', '평균': '5,406 만원', '최대': '6,948만원'}},
 {'기업명': '(주)클로봇',
  '업력': '업력 9년차',
  '업종': '응용 소프트웨어 개발 및 공급업',
  '기업형태': '스타트업',
  '사원수': '136 명',
  '매출액': '134억 4,336만원',
  '주소': '경기 성남시 분당구 성남대로925번길 16, 7층 701호',
  '연봉정보': {'최소': '3,462만원', '평균': '6,605 만원', '최대': '7,915만원'}},
 {'기업명': '휴림로봇(주)',
  '업력': '업력 27년차',
  '업종': '산업용 로봇 제조업',
  '기업형태': '63 명',
  '사원수': '비공개',
  '매출액': '비공개',
  '주소': '충남 천안시 서북구 직산읍 4산단6길 27',
  '연봉정보': {'최소': '2,824만원', '평균': '4,268 만원', '최대': '6,216만원'}}]